In [29]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network

# Importing dataset
df = pd.read_csv('/Users/enzovillafuerte/Desktop/Python_Projects/Hungary_Euro_Report/Datasets/Hungary-Switzerland.csv')
df.head()

################## Data Cleansing Section ##################

team1, team2 = df.team.unique()
df = df[df['team'] == team1]
subs = df[df['type_display_name'] == 'SubstitutionOff']
subs = subs['minute']
first_sub = subs.min()
df = df[df['minute'] < first_sub].reset_index()
df['passer'] = df['name']
df['receiver'] = df['name'].shift(-1)
df = df[df['type_display_name'] == 'Pass']
df = df[df['outcome_type_display_name'] == 'Successful']
avg_locations = df.groupby('passer').agg({'x':['mean'], 'y':['mean', 'count']})
avg_locations.columns = ['x', 'y', 'count']
pass_between = df.groupby(['passer', 'receiver']).id.count().reset_index()
pass_between.rename({'id':'pass_count'}, axis='columns', inplace=True)
pass_between = pass_between.merge(avg_locations, left_on='passer', right_index=True)
pass_between = pass_between.merge(avg_locations, left_on='receiver', right_index=True, suffixes=['', '_end'])


################## Generating Network Section ##################

# NetworkX Graph
G = nx.DiGraph()

for player, data in avg_locations.iterrows():
    G.add_node(player, pos=(data['x'], data['y']), size=data['count'])

for _, row in pass_between.iterrows():
    G.add_edge(row['passer'], row['receiver'], weight=row['pass_count'])

# Pyvis Visualization
net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black')

# Debugging step: Check if nodes and edges are correctly added
print(f"Number of nodes: {len(G.nodes)}")
print(f"Number of edges: {len(G.edges)}")

# Get positions for nodes
pos = {player: (data['x'], data['y']) for player, data in avg_locations.iterrows()}

# Add nodes to Pyvis network
for node in G.nodes:
    x, y = pos[node]
    net.add_node(node, label=node, x=x, y=y, size=G.nodes[node]['size'])
    #print(f"Added node: {node} at position ({x}, {y})")

# Add edges to Pyvis network
for edge in G.edges:
    net.add_edge(edge[0], edge[1], value=G.edges[edge]['weight'])
    #print(f"Added edge from {edge[0]} to {edge[1]} with weight {G.edges[edge]['weight']}")

# Set the physics layout to False to preserve node positions
net.toggle_physics(False)

# Generate and save the network as HTML
#net.show('pass_network.html')

# Generate and save the network as HTML without opening it automatically
net.write_html('pass_network.html')

print("The HTML file has been generated. Open 'pass_network.html' in your browser to view it.")

Number of nodes: 11
Number of edges: 67
The HTML file has been generated. Open 'pass_network.html' in your browser to view it.


In [32]:
avg_locations

,x,y,count
passer,,,
András Schäfer,41.017647,65.205882,17
Attila Fiola,45.560000,17.880000,5
Attila Szalai,42.386364,82.236364,22
Barnabás Varga,58.075000,42.950000,4
Dominik Szoboszlai,53.982353,61.647059,17
Milos Kerkez,56.340000,84.120000,5
Péter Gulácsi,12.800000,48.200000,10
Roland Sallai,47.887500,63.618750,16
Willi Orbán,34.681818,40.990909,22


In [36]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network

# Importing dataset
df = pd.read_csv('/Users/enzovillafuerte/Desktop/Python_Projects/Hungary_Euro_Report/Datasets/Hungary-Switzerland.csv')
df.head()

# Data Preparation (assuming df is already defined)
team1, team2 = df.team.unique()
df = df[df['team'] == team1]
subs = df[df['type_display_name'] == 'SubstitutionOff']
subs = subs['minute']
first_sub = subs.min()
df = df[df['minute'] < first_sub].reset_index()
df['passer'] = df['name']
df['receiver'] = df['name'].shift(-1)
df = df[df['type_display_name'] == 'Pass']
df = df[df['outcome_type_display_name'] == 'Successful']
avg_locations = df.groupby('passer').agg({'x':['mean'], 'y':['mean', 'count']})
avg_locations.columns = ['x', 'y', 'count']
pass_between = df.groupby(['passer', 'receiver']).id.count().reset_index()
pass_between.rename({'id':'pass_count'}, axis='columns', inplace=True)
pass_between = pass_between.merge(avg_locations, left_on='passer', right_index=True)
pass_between = pass_between.merge(avg_locations, left_on='receiver', right_index=True, suffixes=['', '_end'])

# NetworkX Graph
G = nx.DiGraph()

for player, data in avg_locations.iterrows():
    G.add_node(player, pos=(data['x'], data['y']), size=data['count'])

for _, row in pass_between.iterrows():
    G.add_edge(row['passer'], row['receiver'], weight=row['pass_count'])

# Pyvis Visualization
net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black')

# Get positions for nodes
pos = {player: (data['x'], data['y']) for player, data in avg_locations.iterrows()}

# Define the soccer field dimensions (predefined axis)
field_width = 8
field_height = 12

# Add nodes to Pyvis network
for node in G.nodes:
    x, y = pos[node]
    net.add_node(node, label=node, x=x, y=y, size=G.nodes[node]['size'], title=node)
    print(f"Added node: {node} at position ({x}, {y})")

# Add edges to Pyvis network
for edge in G.edges:
    net.add_edge(edge[0], edge[1], value=G.edges[edge]['weight'])
    print(f"Added edge from {edge[0]} to {edge[1]} with weight {G.edges[edge]['weight']}")

# Disable physics to preserve node positions
net.toggle_physics(False)

# Generate and save the network as HTML
net.write_html('pass_network.html')

print("The HTML file has been generated. Open 'pass_network.html' in your browser to view it.")


Added node: András Schäfer at position (41.01764705882353, 65.20588235294117)
Added node: Attila Fiola at position (45.56, 17.88)
Added node: Attila Szalai at position (42.38636363636363, 82.23636363636363)
Added node: Barnabás Varga at position (58.075, 42.95)
Added node: Dominik Szoboszlai at position (53.98235294117647, 61.64705882352941)
Added node: Milos Kerkez at position (56.339999999999996, 84.12)
Added node: Péter Gulácsi at position (12.8, 48.2)
Added node: Roland Sallai at position (47.887499999999996, 63.61875)
Added node: Willi Orbán at position (34.68181818181818, 40.99090909090909)
Added node: Ádám Lang at position (34.661538461538456, 12.26923076923077)
Added node: Ádám Nagy at position (45.26086956521739, 50.38260869565217)
Added edge from András Schäfer to András Schäfer with weight 1
Added edge from András Schäfer to Attila Szalai with weight 1
Added edge from András Schäfer to Barnabás Varga with weight 1
Added edge from András Schäfer to Dominik Szoboszlai with wei

In [37]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network

# Importing dataset
df = pd.read_csv('/Users/enzovillafuerte/Desktop/Python_Projects/Hungary_Euro_Report/Datasets/Hungary-Switzerland.csv')
df.head()

# Assuming df is already defined
team1, team2 = df.team.unique()
df = df[df['team'] == team1]
subs = df[df['type_display_name'] == 'SubstitutionOff']
subs = subs['minute']
first_sub = subs.min()
df = df[df['minute'] < first_sub].reset_index()
df['passer'] = df['name']
df['receiver'] = df['name'].shift(-1)
df = df[df['type_display_name'] == 'Pass']
df = df[df['outcome_type_display_name'] == 'Successful']
avg_locations = df.groupby('passer').agg({'x':['mean'], 'y':['mean', 'count']})
avg_locations.columns = ['x', 'y', 'count']
pass_between = df.groupby(['passer', 'receiver']).id.count().reset_index()
pass_between.rename({'id':'pass_count'}, axis='columns', inplace=True)
pass_between = pass_between.merge(avg_locations, left_on='passer', right_index=True)
pass_between = pass_between.merge(avg_locations, left_on='receiver', right_index=True, suffixes=['', '_end'])

# Define the soccer field dimensions (predefined axis)
field_width = 100
field_height = 100

# Adjust positions to fit within the field dimensions
avg_locations['x'] = avg_locations['x'] * field_width / 100
avg_locations['y'] = avg_locations['y'] * field_height / 100

# NetworkX Graph
G = nx.DiGraph()

for player, data in avg_locations.iterrows():
    G.add_node(player, pos=(data['x'], data['y']), size=data['count'])

for _, row in pass_between.iterrows():
    G.add_edge(row['passer'], row['receiver'], weight=row['pass_count'])

# Pyvis Visualization
net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black')

# Add nodes to Pyvis network
for node, data in G.nodes(data=True):
    x, y = data['pos']
    net.add_node(node, label=node, x=x, y=y, size=data['size'], title=node)

# Add edges to Pyvis network
for edge in G.edges(data=True):
    net.add_edge(edge[0], edge[1], value=edge[2]['weight'])

# Disable physics to preserve node positions
net.toggle_physics(False)

# Generate and save the network as HTML
net.write_html('pass_network.html')

print("The HTML file has been generated. Open 'pass_network.html' in your browser to view it.")


The HTML file has been generated. Open 'pass_network.html' in your browser to view it.
